In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 # 🎓 Capstone Project: Adaptive Spaced Repetition Tutor (SRT)
 ## Phase 1: The Agentic Memory Foundation

**Project Overview:**

This agent solves the "forgetting curve" problem for students. It ingests study materials (summarized by NotebookLM), consolidates them into atomic facts, and schedules them for adaptive review.

**Phase 1 Goals:**
1.  Setup the ADK environment.
2.  Define the `KnowledgeFact` data structure.
3.  Build the `MemoryConsolidationAgent` to extract facts from raw text.
4.  Initialize the `MemoryService` to store these facts.

## ⚙️ Section 1: Setup & Configuration

First, we install the Google Agent Development Kit (ADK) and configure our secure access credentials.

In [ ]:
# 1.1 Install ADK

!pip install google-adk pydantic --quiet
print("✅ ADK installed successfully.")

In [ ]:
# 1.2 Configure API Key (Securely)

import os
from kaggle_secrets import UserSecretsClient

try:
    # Retrieve the API key from Kaggle Secrets
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"⚠️ Authentication Warning: {e}")
    print("If running locally, ensure GOOGLE_API_KEY is set in your environment.")

In [ ]:
# 1.3 Import ADK Components

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import ToolContext
from google.genai import types
from typing import List, Dict, Any
from pydantic import BaseModel, Field
import json
import datetime

print("✅ Imports complete.")

In [ ]:
# 1.4 Configuration & Best Practices
# We use a robust retry configuration to handle potential API timeouts/errors.
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

# Model Configuration
MODEL_NAME = "gemini-2.5-flash-lite" # Cost-effective and fast for extraction
APP_NAME = "SRT_Tutor_App"
USER_ID = "student_user"

print("✅ Configuration ready.")

## 🧠 Section 2: Defining the "Knowledge" Structure

Unlike a simple chat bot, our tutor needs structured data. We use `Pydantic` to define exactly what a "Memory Fact" looks like. This ensures our agent produces consistent data we can query later.

In [ ]:
class KnowledgeFact(BaseModel):
    """Schema for a single unit of knowledge to be memorized."""
    topic: str = Field(..., description="The broad subject (e.g., 'Deep Learning', 'Python').")
    concept: str = Field(..., description="The specific concept name (e.g., 'Dropout', 'List Comprehension').")
    definition: str = Field(..., description="A concise, one-sentence definition or explanation.")
    confidence: str = Field("High", description="Current confidence level: High, Medium, or Low.")
    last_reviewed: str = Field(..., description="Date string YYYY-MM-DD.")

## 🛠️ Section 3: Custom Tools for Memory Ingestion

We need a custom tool that the Agent can call to "save" what it learns. This tool takes the extracted facts and puts them into our `MemoryService`.


In [ ]:
# Initialize the Memory Service (Simulating Long-Term Persistence)
# In production, you might use VertexAiMemoryBankService, but InMemory is perfect for the Capstone demo.

memory_service = InMemoryMemoryService()

def save_knowledge_facts(tool_context: ToolContext, facts: List[Dict[str, Any]]) -> str:
    """
    Saves a list of extracted knowledge facts into the user's long-term memory.
    
    Args:
        facts: A list of dictionaries, where each dictionary represents a KnowledgeFact 
               (topic, concept, definition, confidence, last_reviewed).
    """
    print(f"\n💾 TOOL CALL: Saving {len(facts)} new facts to memory...")
    
    for fact_data in facts:
        # In a real app, we might use memory_service.add_memory(fact) 
        # Here, we simulate it by storing it in the session state or a global store
        # for immediate retrieval demonstration.
        
        # We can leverage the tool_context state to persist this for the session
        if "knowledge_base" not in tool_context.state:
            tool_context.state["knowledge_base"] = []
            
        tool_context.state["knowledge_base"].append(fact_data)
        print(f"   - Saved: {fact_data['concept']} ({fact_data['topic']})")
        
    return f"Successfully saved {len(facts)} facts to long-term memory."

print("✅ Custom Tool `save_knowledge_facts` defined.")

## 🤖 Section 4: The Memory Consolidation Agent

This agent acts as the "Librarian." It takes raw, unstructured text (like your NotebookLM summary) and uses the LLM to structure it into the `KnowledgeFact` format before calling the save tool.


In [ ]:
consolidation_agent = LlmAgent(
    name="MemoryConsolidator",
    model=Gemini(model=MODEL_NAME, retry_options=retry_config),
    instruction=f"""
    You are an expert Knowledge Consolidator. Your goal is to help a student memorize complex topics.
    
    INPUT: You will receive raw study notes or summaries.
    
    TASK:
    1. Analyze the text and extract key concepts, definitions, and facts.
    2. Transform these into a structured list of facts.
    3. For each fact, assign a 'topic', 'concept', and a clear 'definition'.
    4. Set 'confidence' to 'High' (default) and 'last_reviewed' to today's date.
    5. You MUST use the `save_knowledge_facts` tool to store these facts.
    
    Do not just reply with text. You must CALL THE TOOL to save the data.
    """,
    tools=[save_knowledge_facts]
)

print("✅ Consolidation Agent initialized.")

## 🚀 Section 5: Running Phase 1
 
Let's simulate the workflow. Imagine you just finished studying "Gradient Descent" and you have a summary from NotebookLM. You paste it here, and the Agent ingests it.


In [ ]:
# 1. Setup the Runner
# We use InMemorySessionService for the conversation flow
session_service = InMemorySessionService()
runner = Runner(
    agent=consolidation_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service
)

# 2. The Input (Simulated output from NotebookLM)
raw_study_notes = """
Summary of Deep Learning Optimization:
Gradient Descent is an optimization algorithm used to minimize some function by iteratively moving in the 
direction of steepest descent as defined by the negative of the gradient.
The Learning Rate is a hyperparameter that controls how much to change the model in response to the 
estimated error each time the model weights are updated. If it's too small, training is slow. If too large, 
it might overshoot the minimum.
Stochastic Gradient Descent (SGD) performs a parameter update for each training example, unlike Batch GD 
which uses the whole dataset.
"""

# 3. Run the Agent
print(f"user > Processing study notes... (Length: {len(raw_study_notes)} chars)\n")

# --- FIX: Create the session explicitly first ---
session_id = "ingestion_session_01"
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=session_id)

# We use a loop to process the stream, matching the assignment notebook style
async for event in runner.run_async(
    user_id=USER_ID,
    session_id=session_id,
    new_message=types.Content(role="user", parts=[types.Part(text=raw_study_notes)])
):
    # We print the tool calls and model responses as they happen
    if event.content and event.content.parts:
        for part in event.content.parts:
            if part.text:
                print(f"agent > {part.text}")
            if part.function_call:
                print(f"🛠️ Agent is calling tool: {part.function_call.name}")

### ✅ Phase 1 Complete
 
 If you see the "TOOL CALL: Saving..." output above, your agent has successfully:
 1.  **Reasoned** over raw text.
 2.  **Structured** it into specific data formats.
 3.  **Executed** a custom tool to persist that knowledge.
 
 This completes the foundation. In Phase 2, we will build the **Assessment Agent** that retrieves these facts to quiz you!